In [75]:
from string import ascii_uppercase as abc_upper
from string import ascii_lowercase as abc_lower
import math

def hill_encrypt(msg,key):
    msg = msg.replace(' ','').lower()
    ciphertext = ""
    
    key_dim = int(math.sqrt(len(key)))
    key_mat = [[abc_upper.index(key[j]) for j in range(i*key_dim,i*key_dim+key_dim)] for i in range(key_dim)]
    
    for i in range(0,len(msg),key_dim):
        block = msg[i:i+key_dim]
        block_mat = []
        for char in block:
            block_mat.append([abc_lower.index(char)])
        encrypted = multiply_matrices(key_mat,block_mat)
        for row in encrypted:
            ciphertext+=abc_upper[row[0]]
    return add_spaces(ciphertext)
            

In [76]:
def hill_decrypt(ciphertext,key):
    cipher = ciphertext.replace(' ','')
    msg = ""
    
    key_dim = int(math.sqrt(len(key)))
    key_mat = gaussian_elimination([[abc_upper.index(key[j]) for j in range(i*key_dim,i*key_dim+key_dim)] for i in range(key_dim)],
                                   gen_identity_matrix(key_dim)
                                  )
    
    for i in range(0,len(cipher),key_dim):
        block = cipher[i:i+key_dim]
        block_mat = []
        for char in block:
            block_mat.append([abc_upper.index(char)])
        decrypted = multiply_matrices(key_mat,block_mat)
        for row in decrypted:
            msg+=abc_lower[row[0]]
    return msg

In [77]:
#multiplies 2 matrices mod 26
def multiply_matrices(a,b):
    if len(a[0])!=len(b):
        raise Exception("# of cols in A must match # of rows in B",len(a[0]),len(b))
        
    new_mat = [[0]*len(b[0]) for i in range(len(a))]
    for i in range(len(a)):
        for j in range(len(b[0])):
            new_mat[i][j]=sum([a[i][k]*b[k][j] for k in range(len(a))])%26
    return new_mat       

In [97]:
#https://en.wikipedia.org/wiki/Gaussian_elimination
#https://online.stat.psu.edu/statprogram/reviews/matrix-algebra/gauss-jordan-elimination

#inverts matrix by using row reduction
#[a1 a2 a3 1 0 0]    [1][0][0][a1'][a2'][a3']
#[b1 b2 b3 0 1 0] -->[0][1][0][b1'][b2'][b3'] 
#[c1 c2 c3 0 0 1]    [0][0][1][c1'][c2'][b3']
#where ' means that the element is in the matrix's inverse

#1: set pivot row, r, and column, c, to 0
#2: swap pivot row with the row of the highest absolute value mod 26 in that column. Must also be relatively prime to modulus
#3: multiply the pivot row by a constant k s.t k*m[r][c] = 1 (mod 26)
#4: loop over all rows i below pivot, adding a multiple k of the pivot row s.t m[i][c]-m[r][c]*k = 0 (mod 26)
#5: loop over all remaining 0s starting from top left

#3 elementary functions are defined at the bottom

def gaussian_elimination(m,augmentation=[]):
    d=0
    if len(augmentation)!=0:
        for i in range(len(m)):
            m[i]+=augmentation[i]
    
    r=c=0
    while r<len(m) and c<(len(m[0])-len(augmentation[0])):
        
        #pivot is max index that is relatively prime to 26
        pivot=get_max_idx(m,r,c)
        
        #this shouldn't ever happen but I will leave it for now
        if m[pivot][c]==0:
            c+=1
        else:
            #print("pivot",pivot)
            print_matrix(m)
            m=swap_rows(m,r,pivot)
            
            #actually shocked that this works
            #if none of the elements are relatively prime, subtract row below it, then repeat the process. 
            #This probably doesn't always work
            gcd,p,q=gcde(m[r][c],26)
            if gcd!=1 and r+1<len(m):
                s=get_max_idx(m,r+1,c)
                m=add_scalar(m,r,s,-1)
                gcd,p,q=gcde(m[r][c],26)   
            m=mul_row(m,r,q)
            
            #change rows below to 0
            for h in range(r+1,len(m)):
                if m[h][c]!=0:
                    m=add_scalar(m,h,r,-m[h][c])
        r+=1
        c+=1
    
    #go through the numbers to the right of the diagonal and change them all to zeros, by subtracting the row that corresponds with the column
    #for example, to turn m[i][j] into 0, subtract m[j], because that row contains a 1 at index j
    #so you can just subtract m[i]-m[i]*m[j]. ALl cols !=j have 0s, so only element i is being affected in the scaled row
    for r in range(len(m)):
        for c in range(r+1,len(m[0])-len(augmentation[0])):
            if m[r][c]!=0:
                m=add_scalar(m,r,c,-m[r][c])
        
    return [r[len(augmentation):] for r in m]


def add_scalar(m,row,scalar,k):
    for i in range(len(m[row])):
        m[row][i]=(m[row][i]+m[scalar][i]*k)%26
    return m
    
def mul_row(m,row,k):
    for i in range(len(m[row])):
        m[row][i]=(m[row][i]*k)%26
    return m
    
def swap_rows(m,r1,r2):
    m[r1],m[r2]=m[r2],m[r1]
    return m
                           
#matrix=[[2,-1,0],[-1,2,-1],[0,-1,2]]
#mat=[[0,1,2],[3,4,6],[8,1,17]]
#mat2=[[19,0,13],[6,4,17],[8,13,4]]

#a=invert_matrix(mat2)
#print_matrix(a)

In [79]:
#matrix=[[2,-1,0,1,0,0],[-1,2,-1,0,1,0],[0,-1,2,0,0,1]]
matrix=[[19,0,13,1,0,0],[6,4,7,0,1,0],[8,13,4,0,0,1]]
#funcs=[add_scalar,mul_row,swap_rows]
def manual_matrix_ops(matrix,augmentation):
    if len(augmentation)!=0:
        for i in range(len(matrix)):
            matrix[i]+=augmentation[i]
    while True:
        print_matrix(matrix)
        print()
        print("Choose operation:")
        print("0-add scalar")
        print("1-mul_row")
        print("2-swap_rows")
        op=int(input())
        if op==0:
            r=int(input("Input row to add to:"))
            s=int(input("Input row to add"))
            k=eval(input("Input multiple of s to add"))
            matrix=add_scalar(matrix,r,s,k)
        elif op==1:
            r=int(input("Input row to scale"))
            k=eval(input("Input scale factor"))
            matrix=mul_row(matrix,r,k)
        elif op==2:
            r1=int(input("Input first row to swap"))
            r2=int(input("Input second row to swap"))
            matrix=swap_rows(matrix,r1,r2)

In [80]:
#gets the largest value, relatively prime to 26 (if there is one), in the specified column of the matrix, below the pivot row
def get_max_idx(m,r,c):
    max_pivot=0
    max_prime_pivot=0
    max_idx=r
    max_prime_idx=r
    for i in range(r,len(m)):
        g=gcde(m[i][c],26)[0]
        if g==1 and m[i][c]%26>max_prime_pivot:
            max_prime_pivot=m[i][c]%26
            max_prime_idx=i
        if m[i][c]%26>max_pivot:
            max_pivot=m[i][c]%26
            max_idx=i
    if max_prime_pivot==0:
        return max_idx
    else:
        return max_prime_idx

In [81]:
#generates nxn identity matrix
def gen_identity_matrix(n):
    mat = [[0]*n for i in range(n)]
    for i in range(n):
        mat[i][i]=1
    return mat     

In [82]:
#prints matrix nicely
def print_matrix(m):
    print()
    for i in m:
        out="|"
        for j in i:
            #j=int(j)
            if j>=0 and j<10:
                out+=" "
            out+=" "+str(j)
        out+=" |"
        print(out)

In [104]:
#probably won't need this but
def get_determinant(m):
    return None

In [84]:
#add space every 5 places for ciphertext
def add_spaces(ciphertext):
    ciphertext=" ".join([ciphertext[i:i+5] for i in range(0,len(ciphertext),5)])
            
    return ciphertext

In [539]:
hill_encrypt("Why sometimes Ive believed as many as six impossible things before breakfastz".lower(),"PORCUPINE")

[[15, 14, 17], [2, 20, 15], [8, 13, 4]]
EYZUC KUOZW KMOCP YNANV SYQTW QOLMA IWULW YZIKJ DSYXI WFEVX AEGXQ ZKZKG MQPZL X


In [564]:
hill_decrypt("EYZUC KUOZW KMOCP YNANV SYQTW QOLMA IWULW YZIKJ DSYXI WFEVX AEGXQ ZKZKG MQPZL X","PORCUPINE")


|  15  14  17  1  0  0 |
|  2  20  15  0  1  0 |
|  8  13  4  0  0  1 |

|  1  20  15  7  0  0 |
|  0  6  11  12  1  0 |
|  0  9  14  22  0  1 |

|  1  20  15  7  0  0 |
|  0  1  16  14  0  3 |
|  0  0  19  6  1  8 |
whysometimesivebelievedasmanyassiximpossiblethingsbeforebreakfastz


In [85]:
def gcde(a,b):
    if b==0:
        return a,0,1
    else:
        #print('{a} = {b} * {q} + {r}'.format(a=a,b=b,q=math.floor(a/b),r=a%b))
        g,m,n = gcde(b,a%b)
        return g, n-(a//b)*m, m

In [690]:
mat=[[19,7,0,0],
     [0,0,19,7],
     [0,19,0,0],
     [0,0,0,19]]
aug=[[1],
    [15],
    [24],
    [11]]
mat2=[[0,1,2],[3,4,6],[8,1,17]]
#gaussian_elimination(mat,aug)
gaussian_elimination(mat2,gen_identity_matrix(3))
#manual_matrix_ops(mat,aug)


|  0  1  2  1  0  0 |
|  3  4  6  0  1  0 |
|  8  1  17  0  0  1 |

|  1  10  2  0  9  0 |
|  0  1  2  1  0  0 |
|  0  25  1  0  6  1 |

|  1  10  2  0  9  0 |
|  0  1  25  0  20  25 |
|  0  0  3  1  6  1 |


[[22, 19, 6], [9, 22, 8], [9, 2, 9]]

In [101]:
def get_key_from_blocks(c_blocks,pt_blocks):

    abc=abc_lower

    block_len=len(c_blocks[0])
    mat=[[0 for j in range(block_len**2)] for i in range(block_len**2)]
    aug=[[0] for i in range(block_len**2)]

    for i in range(block_len):
        for j in range(block_len):
            for k in range(block_len):
                mat[i*block_len+j][j*block_len+k]=abc.index(pt_blocks[i][k].lower())
            l=abc.index(c_blocks[i][j].lower())
            aug[block_len*i+j][0]=l
    
    #key=gaussian_elimination(mat,aug)
    return mat,aug
    #return ''.join([abc[ch[0]] for ch in key]).upper()

In [89]:
def crack_hill_cipher(cipher,c_blocks,pt_blocks):
    key=get_key_from_blocks(c_blocks,pt_blocks)
    msg=hill_decrypt(cipher,key)
    return key,msg

In [90]:
c_blocks=['RCS','XLB','CFC']
pt_blocks=['nec','tar','ine']
msg="VCKUP EDOPS JICJP NBZCV DDMKI IRQKP WAKQI QMJEX HSQAH XHSZX LCTC"
print(crack_hill_cipher(msg,c_blocks,pt_blocks))

('JACKFRUIT', 'beginatthebeginningandgoontillyoucometotheendthenstopq')


In [91]:
msg="""Whysometimes Ive believed as many as six impossible things before breakfastz"""
key="PORCUPINE"
print(msg)
print(hill_encrypt(msg,key))

Whysometimes Ive believed as many as six impossible things before breakfastz
EYZUC KUOZW KMOCP YNANV SYQTW QOLMA IWULW YZIKJ DSYXI WFEVX AEGXQ ZKZKG MQPZL X


In [688]:
key="PORCUPINE"
key_dim = int(math.sqrt(len(key)))
key_mat = [[abc_upper.index(key[j]) for j in range(i*key_dim,i*key_dim+key_dim)] for i in range(key_dim)]
print(key_mat)

[[15, 14, 17], [2, 20, 15], [8, 13, 4]]


In [92]:
abc=abc_lower
def word_to_nums(word):
    return [abc.index(ch) for ch in word.replace(' ','').lower()]

In [32]:
print(word_to_nums('RCSXLBCFC'))
print(word_to_nums('nectarine'))
#c_blocks=['RCS','XLB','CFC']
#pt_blocks=['nec','tar','ine']

[17, 2, 18, 23, 11, 1, 2, 5, 2]
[13, 4, 2, 19, 0, 17, 8, 13, 4]


In [67]:
c_blocks=['RCS','XLB','CFC']
pt_blocks=['nec','tar','ine']
m,aug=get_key_from_blocks(c_blocks,pt_blocks)
print_matrix(m)


| 13  4  2  0  0  0  0  0  0 |
|  0  0  0 13  4  2  0  0  0 |
|  0  0  0  0  0  0 13  4  2 |
| 19  0 17  0  0  0  0  0  0 |
|  0  0  0 19  0 17  0  0  0 |
|  0  0  0  0  0  0 19  0 17 |
|  8 13  4  0  0  0  0  0  0 |
|  0  0  0  8 13  4  0  0  0 |
|  0  0  0  0  0  0  8 13  4 |


In [68]:
m=swap_rows(m,0,3)
print_matrix(m)
print(get_max_idx(m,1,1))


| 19  0 17  0  0  0  0  0  0 |
|  0  0  0 13  4  2  0  0  0 |
|  0  0  0  0  0  0 13  4  2 |
| 13  4  2  0  0  0  0  0  0 |
|  0  0  0 19  0 17  0  0  0 |
|  0  0  0  0  0  0 19  0 17 |
|  8 13  4  0  0  0  0  0  0 |
|  0  0  0  8 13  4  0  0  0 |
|  0  0  0  0  0  0  8 13  4 |
6


In [98]:
c_blocks=['AHW','ZTF','LRZ']
pt_blocks=['and','but','for']
msg="ZWZII MBBSQ KMKNN QKYKM ZZWZU ALXPN QKYKM ZZWZI IMBBS QKMAA A"
print(crack_hill_cipher(msg,c_blocks,pt_blocks))


|  0 13  3  0  0  0  0  0  0  0 |
|  0  0  0  0 13  3  0  0  0  7 |
|  0  0  0  0  0  0  0 13  3 22 |
|  1 20 19  0  0  0  0  0  0 25 |
|  0  0  0  1 20 19  0  0  0 19 |
|  0  0  0  0  0  0  1 20 19  5 |
|  5 14 17  0  0  0  0  0  0 11 |
|  0  0  0  5 14 17  0  0  0 17 |
|  0  0  0  0  0  0  5 14 17 25 |

|  1  8 19  0  0  0  0  0  0 23 |
|  0  0  0  0 13  3  0  0  0  7 |
|  0  0  0  0  0  0  0 13  3 22 |
|  0 12  0  0  0  0  0  0  0  2 |
|  0  0  0  1 20 19  0  0  0 19 |
|  0  0  0  0  0  0  1 20 19  5 |
|  0 13  3  0  0  0  0  0  0  0 |
|  0  0  0  5 14 17  0  0  0 17 |
|  0  0  0  0  0  0  5 14 17 25 |

|  1  8 19  0  0  0  0  0  0 23 |
|  0  1  3  0  0  0  0  0  0 24 |
|  0  0  0  0  0  0  0 13  3 22 |
|  0  0 16  0  0  0  0  0  0  0 |
|  0  0  0  1 20 19  0  0  0 19 |
|  0  0  0  0  0  0  1 20 19  5 |
|  0  0  0  0 13  3  0  0  0  7 |
|  0  0  0  5 14 17  0  0  0 17 |
|  0  0  0  0  0  0  5 14 17 25 |

|  1  8 19  0  0  0  0  0  0 23 |
|  0  1  3  0  0  0  0  0  0 24 |
|  0  0  2

In [103]:
print(word_to_nums('AHWZTFLRZ'))
print(word_to_nums('andbutfor'))
print_matrix(get_key_from_blocks(c_blocks,pt_blocks)[0])
c_blocks=['AHW','ZTF','LRZ']
pt_blocks=['and','but','for']

[0, 7, 22, 25, 19, 5, 11, 17, 25]
[0, 13, 3, 1, 20, 19, 5, 14, 17]

|  0 13  3  0  0  0  0  0  0 |
|  0  0  0  0 13  3  0  0  0 |
|  0  0  0  0  0  0  0 13  3 |
|  1 20 19  0  0  0  0  0  0 |
|  0  0  0  1 20 19  0  0  0 |
|  0  0  0  0  0  0  1 20 19 |
|  5 14 17  0  0  0  0  0  0 |
|  0  0  0  5 14 17  0  0  0 |
|  0  0  0  0  0  0  5 14 17 |
